## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_0 = pd.read_csv("../../바이너리_최종데이터_1004/gcc8/o0/o0_bincore8.csv", index_col=0)
print(bin8_0.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_0.reset_index(inplace=True, drop=True)

print('shape -', bin8_0.shape)
print('reset_index 완료')
print('input data shape')
bin8_0.head()

(30111221, 2)
shape - (30111221, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_0['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_0['label'].value_counts())

256
0    30060985
1       50236
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin8_0[bin8_0['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 16 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_0), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_0', len(ls_idx_bin))

# loc 로 수정필요
bin8_0_Ngram = bin8_0.loc[ls_idx_bin,:].copy()

803776
나머지 0
최종 길이 803776
bin8_0 803776


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_0_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_0.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_0)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_0[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

50236.0
0 12831159
1000 25480144
2000 21182650
3000 15807945
4000 27602937
5000 15913005
6000 26689639
7000 1870948
8000 16840029
9000 12438548
10000 14645830
11000 29398413
12000 8776659
13000 12956364
14000 5068380
15000 1670329
16000 26829982
17000 15708976
18000 25984559
19000 29783138
20000 16463377
21000 17740854
22000 18738957
23000 22342610
24000 27311939
25000 21305429
26000 8693357
27000 14961777
28000 25396489
29000 17474197
30000 17821842
31000 24809955
32000 13723323
33000 10905561
34000 27355458
35000 3281468
36000 21429508
37000 25058636
38000 28943948
39000 14829117
40000 30053917
41000 17463454
42000 25519807
43000 7655243
44000 5198179
45000 4416952
46000 19384706
47000 25790720
48000 11439180
49000 5684485
50000 29938957
완료
50236


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_0['label'].value_counts()[1])

50236
50236


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_0_Ngram])
final.shape

(1607552, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_0_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_0_onehot_Ngram = pd.concat([final['label'], bc8_0_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_0_onehot_Ngram.shape)

원핫인코딩완료
(1607552, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc8_0 = bc8_0_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_0 = bc8_0_onehot_Ngram['label'].to_numpy()
print(x_bc8_0.shape, x_bc8_0.shape)

x_bc8_0 = x_bc8_0.reshape(-1, right_idx, x_bc8_0.shape[1])
y_bc8_0 = y_bc8_0.reshape(-1, right_idx, 1)

print(x_bc8_0.shape, y_bc8_0.shape)

(1607552, 256) (1607552, 256)
(100472, 16, 256) (100472, 16, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_0.shape[0])

x_bc8_0 = x_bc8_0[p]
y_bc8_0 = y_bc8_0[p]

print(x_bc8_0.shape, y_bc8_0.shape)

(100472, 16, 256) (100472, 16, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(24, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습

In [11]:
model1 = models.Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

print('======Training stage======')
model1.fit(x_bc8_0,
           y_bc8_0,
           epochs = 500,
           batch_size = 32,
           validation_split=0.3,
           callbacks=[early_stopping])

model1.save('gcc8_bin_core_s16_h24_o0.h5')
print('save 완료')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 70330 samples, validate on 30142 samples
Epoch 1/500
70330/70330 [==============================] - 23s 321us/sample - loss: 0.0119 - accuracy: 0.9974 - val_loss: 5.3737e-04 - val_accuracy: 0.9999
Epoch 2/500
70330/70330 [==============================] - 15s 219us/sample - loss: 3.5609e-04 - accuracy: 0.9999 - val_loss: 5.1685e-04 - val_accuracy: 0

save 완료
